In [1]:
import pandas as pd
import pyspark
from datetime import datetime
from window_generator import generate_windows_sc as wg
from datetime import timedelta

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [2]:
auctionsRDD = sc.textFile("./data/auctions.csv", 8).map(lambda line: line.split(",")).filter(lambda x: x[0] != 'date')
auctionsRDD.take(5)

[['2019-04-23 18:58:00.842116', '2564673204772915246', '1', '0'],
 ['2019-04-23 18:58:01.530771', '4441121667607578179', '7', '0'],
 ['2019-04-23 18:58:01.767562', '7721769811471055264', '1', '0'],
 ['2019-04-23 18:58:02.363468', '6416039086842158968', '1', '0'],
 ['2019-04-23 18:58:02.397559', '1258642015983312729', '1', '0']]

In [3]:
auctions = auctionsRDD.map(lambda x: (x[1], pd.to_datetime(x[0]) ) ) 
#device_id, date

In [4]:
window1, window2, window3, window4, window5, window6, window7 = wg(auctions)

In [5]:
window1 = window1.reduceByKey(lambda x,y: min(x,y))
window2 = window2.reduceByKey(lambda x,y: min(x,y))
window3 = window3.reduceByKey(lambda x,y: min(x,y))
window4 = window4.reduceByKey(lambda x,y: min(x,y))
window5 = window5.reduceByKey(lambda x,y: min(x,y))
window6 = window6.reduceByKey(lambda x,y: min(x,y))
window7 = window7.reduceByKey(lambda x,y: min(x,y))

In [6]:
INITIAL_DATE = pd.to_datetime('2019-04-20 00:00:00.00')
INITIAL_DATE_V2 = pd.to_datetime(INITIAL_DATE + timedelta(days=1))
INITIAL_DATE_V3 = pd.to_datetime(INITIAL_DATE + timedelta(days=2))
INITIAL_DATE_V4 = pd.to_datetime(INITIAL_DATE + timedelta(days=3))
INITIAL_DATE_V5 = pd.to_datetime(INITIAL_DATE + timedelta(days=4))
INITIAL_DATE_V6 = pd.to_datetime(INITIAL_DATE + timedelta(days=5))
INITIAL_DATE_V7 = pd.to_datetime(INITIAL_DATE + timedelta(days=6))

window1 = window1.map(lambda x: (x[0], (x[1] - INITIAL_DATE).total_seconds() ) )
window2 = window2.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V2).total_seconds() ) )
window3 = window3.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V3).total_seconds() ) )
window4 = window4.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V4).total_seconds() ) )
window5 = window5.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V5).total_seconds() ) )
window6 = window6.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V6).total_seconds() ) )
window7 = window7.map(lambda x: (x[0], (x[1] - INITIAL_DATE_V7).total_seconds() ) )

window1.take(10)

[('1775327097246149922', 8357.932903),
 ('6209341105994526611', 17401.462635),
 ('3007879526340590519', 11142.718479),
 ('4244907993260948496', 68164.578006),
 ('5881241422551377863', 22555.322438),
 ('6671154453476588515', 171905.094635),
 ('7629923463948763367', 249569.5034),
 ('4352282730343079347', 6709.182927),
 ('5988944724362867569', 8584.337953),
 ('4531465131835661106', 189833.244538)]

In [7]:
def toCSVLine(data):
  return (','.join(str(d) for d in data))

lines = window1.map(toCSVLine)
lines.saveAsTextFile('./data/auctions/av1.csv')

In [8]:
lines2 = window2.map(toCSVLine)
lines2.saveAsTextFile('./data/auctions/aw2')

lines3 = window3.map(toCSVLine)
lines3.saveAsTextFile('./data/auctions/aw3')

lines4 = window4.map(toCSVLine)
lines4.saveAsTextFile('./data/auctions/aw4')

lines5 = window5.map(toCSVLine)
lines5.saveAsTextFile('./data/auctions/aw5')

lines6 = window6.map(toCSVLine)
lines6.saveAsTextFile('./data/auctions/aw6')

lines7 = window7.map(toCSVLine)
lines7.saveAsTextFile('./data/auctions/aw7')